## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Poronaysk,RU,2021-08-07 02:48:22,49.2167,143.1167,59.00,88,100,8.50,overcast clouds
1,1,Kyren,RU,2021-08-07 02:44:04,51.6828,102.1408,63.03,75,100,1.45,overcast clouds
2,2,Rikitea,PF,2021-08-07 02:48:22,-23.1203,-134.9692,74.77,76,9,10.54,clear sky
3,3,Ushuaia,AR,2021-08-07 02:48:23,-54.8000,-68.3000,33.46,80,75,21.85,light snow
4,4,Mount Gambier,AU,2021-08-07 02:44:05,-37.8333,140.7667,51.71,87,75,17.27,broken clouds


In [5]:
# Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [6]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Mustafakemalpasa,TR,2021-08-07 02:48:23,40.0353,28.4117,77.05,83,66,2.51,broken clouds
9,9,Hami,CN,2021-08-07 02:44:07,42.8000,93.4500,83.16,16,100,5.12,overcast clouds
10,10,Sojat,IN,2021-08-07 02:44:07,25.9167,73.6667,80.17,76,100,6.24,overcast clouds
12,12,Labuhan,ID,2021-08-07 02:44:07,-6.8844,112.2051,87.35,61,24,13.98,few clouds
20,20,Kavaratti,IN,2021-08-07 02:43:57,10.5669,72.6420,81.95,79,96,22.68,heavy intensity rain
23,23,Atuona,PF,2021-08-07 02:44:09,-9.8000,-139.0333,77.88,79,13,18.07,few clouds
25,25,Chaohu,CN,2021-08-07 02:48:27,31.6000,117.8667,89.71,70,100,9.60,overcast clouds
26,26,Lovington,US,2021-08-07 02:48:27,32.9440,-103.3485,82.02,36,1,17.16,clear sky
27,27,San Patricio,US,2021-08-07 02:44:10,28.0170,-97.5169,85.86,74,8,11.63,clear sky
29,29,Xinpu,CN,2021-08-07 02:48:28,34.5997,119.1594,84.65,71,96,3.94,overcast clouds


In [7]:
# Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                252
City                   252
Country                252
Date                   252
Lat                    252
Lng                    252
Max Temp               252
Humidity               252
Cloudiness             252
Wind Speed             252
Current Description    252
dtype: int64

In [8]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Mustafakemalpasa,TR,77.05,broken clouds,40.0353,28.4117,
9,Hami,CN,83.16,overcast clouds,42.8000,93.4500,
10,Sojat,IN,80.17,overcast clouds,25.9167,73.6667,
12,Labuhan,ID,87.35,few clouds,-6.8844,112.2051,
20,Kavaratti,IN,81.95,heavy intensity rain,10.5669,72.6420,
23,Atuona,PF,77.88,few clouds,-9.8000,-139.0333,
25,Chaohu,CN,89.71,overcast clouds,31.6000,117.8667,
26,Lovington,US,82.02,clear sky,32.9440,-103.3485,
27,San Patricio,US,85.86,clear sky,28.0170,-97.5169,
29,Xinpu,CN,84.65,overcast clouds,34.5997,119.1594,


In [12]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = f"{lat},{lng}"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Mustafakemalpasa,TR,77.05,broken clouds,40.0353,28.4117,Erkek Öğrenci Yurdu
9,Hami,CN,83.16,overcast clouds,42.8000,93.4500,Laoyutang Resort
10,Sojat,IN,80.17,overcast clouds,25.9167,73.6667,Mahendi Nagari Sojat City
12,Labuhan,ID,87.35,few clouds,-6.8844,112.2051,PT PPI Lamongan Tambak C
20,Kavaratti,IN,81.95,heavy intensity rain,10.5669,72.6420,Hotels in Lakshadweep Islands
...,...,...,...,...,...,...,...
655,Mingaora,PK,79.16,clear sky,34.7758,72.3625,Burj Al Swat
658,Atambua,ID,83.70,broken clouds,-9.1061,124.8925,Pastori 1 Jemaat Polycarpus
660,Abu Dhabi,AE,89.62,clear sky,24.4667,54.3667,Ramee Garden Hotel Apartments
662,Acarau,BR,79.38,few clouds,-2.8856,-40.1200,castelo pizzaria


In [18]:
# Create the output File (CSV)
output_data_file = " Vacation_Search/WeatherPy_vacation.csv"
# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))